# 라이브러리

In [ ]:
import pandas as pd
import numpy as np
import re 

from tqdm.auto import tqdm
from joblib import parallel_backend
import contextlib


In [ ]:
@contextlib.contextmanager
def tqdm_joblib(tqdm_object):
    from joblib import Parallel
    old_batch_callback = Parallel.BatchCompletionCallBack

    class TqdmBatchCompletionCallBack(old_batch_callback):
        def __call__(self, *args, **kwargs):
            tqdm_object.update(n=self.batch_size)
            return super().__call__(*args, **kwargs)

    Parallel.BatchCompletionCallBack = TqdmBatchCompletionCallBack
    try:
        yield tqdm_object
    finally:
        Parallel.BatchCompletionCallBack = old_batch_callback
        tqdm_object.close()


# 재무정보_final_v2(16~24) 파일 열기
- 기업 정보만을 위해 열어보는 파일
    - 재무 데이터에 등장한 모든 기업
    - 연도별로 살아 있었던 기업
    - 특허 여부와 무관

In [ ]:
financial_df = pd.read_csv('../../data/재무정보_final_v2(16~24)_수정_최종.csv', encoding="utf-8", encoding_errors='replace', engine="python")
# financial_df.head(5)

In [ ]:
financial_df.columns.tolist()

# 특허정보_final 파일 열기

In [ ]:
patent_final = pd.read_csv('../../data/특허정보_final_v2.csv', encoding="cp949", encoding_errors='replace', engine="python")
# patent_final.head(5)

In [ ]:
patent_final = patent_final.rename(columns={
    'ipcNumber_IPC코드': 'ipcNumber',
    'applicationDate_출원일자': 'applicationDate',
    'astrtCont_초록':'astrtCont',
    'applicationNumber_출원번호':'applicationNumber',
     'registerDate_등록일자':'registerDate',
      'indexNo_일련번호':'indexNo',
 'registerStatus_등록상태':'registerStatus',
 'inventionTitle_발명의명칭':'inventionTitle',
 'applicantName_출원인명':'applicantName'



    
})


In [ ]:
patent_final.columns.tolist()

# 잠깐 .......

## 특허 데이터 최소 전처리

In [ ]:
patent_df = patent_final.copy()

# 출원일 datetime 보장
patent_df['applicationDate'] = pd.to_datetime(patent_df['applicationDate'])

# 출원년도 
patent_df['application_year'] = patent_df['applicationDate'].dt.year


In [ ]:
patent_df['application_year'] = pd.to_numeric(
    patent_df['application_year'],
    errors='coerce'
)


In [ ]:
def normalize_company_name(name: str) -> str:
    if pd.isna(name):
        return None

    name = str(name)

    # 1. 소문자 변환
    name = name.lower()

    # 2. 법인 표기 제거
    name = re.sub(r'\(주\)|㈜|주식회사', '', name)

    # 3. 불필요한 공백 제거
    name = re.sub(r'\s+', '', name)

    # 4. 특수문자 제거 (필요 최소)
    name = re.sub(r'[^\w가-힣]', '', name)

    return name


In [ ]:
#=====================================================================================
# 두 데이터에 동일하게 적용 
#=====================================================================================
financial_df['company_norm'] = financial_df['기업명'].apply(normalize_company_name)
patent_df['company_norm'] = patent_df['company_name'].apply(normalize_company_name)

#=====================================================================================
# 매칭 품질 진단
#=====================================================================================

# 재무 기준 기업 수
financial_companies = set(financial_df['company_norm'].dropna())

# 특허 기준 기업 수
patent_companies = set(patent_df['company_norm'].dropna())

print("재무 기업 수:", len(financial_companies))
print("특허 기업 수:", len(patent_companies))
print("교집합 수:", len(financial_companies & patent_companies))


In [ ]:
company_year_df = (
    financial_df[['company_norm', '연도']]
    .drop_duplicates()
    .sort_values(['company_norm', '연도'])
    .reset_index(drop=True)
)


In [ ]:
# =====================================================
# [STEP 0] 매출액 숫자 변환 (반드시 growth 계산 전에)
# =====================================================

financial_df['매출액'] = (
    financial_df['매출액']
    .astype(str)                 # 혹시 모를 타입 통일
    .str.replace(',', '')        # 천 단위 콤마 제거
    .str.replace(' ', '')        # 공백 제거
)

financial_df['매출액'] = pd.to_numeric(
    financial_df['매출액'],
    errors='coerce'              # 숫자 아닌 건 NaN
)

# =====================================================
# [STEP 1] 정렬 (절대 생략 금지)
# =====================================================
financial_df = financial_df.sort_values(
    ['company_norm', '연도']
)

# =====================================================
# [STEP 2] 전년도 매출 (lag)
# =====================================================
financial_df['revenue_lag_1'] = (
    financial_df
    .groupby('company_norm')['매출액']
    .shift(1)
)

# =====================================================
# [STEP 3] 매출 성장률
# =====================================================
financial_df['revenue_growth'] = (
    (financial_df['매출액'] - financial_df['revenue_lag_1'])
    / financial_df['revenue_lag_1']
)
# =====================================================
# [STEP 4] 성장 여부 라벨
# =====================================================
financial_df['growth_binary'] = (
    financial_df['revenue_growth'] > 0
).astype(int)

# =====================================================
# [STEP 5] 임시 타겟 변수 (절대 성장 여부)
# =====================================================
financial_df['y'] = financial_df['growth_binary']


## 특허 피처 생성 함수

In [ ]:
import pandas as pd
import numpy as np

def make_patent_features(company_norm, t, patent_df):
    """
    company_norm × 연도(t) 기준 특허 피처 생성
    - warm-up 적용: t < 2019 → None
    - 누수 방지: t-1 시점까지만 사용
    """

    # ===============================
    # 0. Warm-up 강제
    # ===============================
    t = int(t)
    if t < 2019:
        return None

    # 기업 기준 필터
    df = patent_df[patent_df['company_norm'] == company_norm].copy()

    # ===============================
    # 1. 기간 필터 (누수 방지 핵심)
    # ===============================
    window_5y = df[
        (df['application_year'] >= t - 4) &
        (df['application_year'] <= t - 1)
    ]

    window_recent = df[
        (df['application_year'] >= t - 2) &
        (df['application_year'] <= t - 1)
    ]

    # # 등록률 코호트: t-4 ~ t-3
    # cohort_register = df[
    #     (df['application_year'] >= t - 4) &
    #     (df['application_year'] <= t - 3)
    # ]

    # 피인용 as-of cutoff
    citation_window = window_5y[
        window_5y['applicationDate'] <= pd.Timestamp(f"{t-1}-12-31")
    ].copy()

    # ===============================
    # 2. 피처 계산
    # ===============================
    patent_count_5y = len(window_5y)
    patent_count_recent = len(window_recent)

    ipc_diversity = (
        window_5y['ipcNumber']
        .dropna()
        .astype(str)
        .str[:4]
        .nunique()
        if patent_count_5y > 0 else 0
    )

    # # 등록률: registerDate 기준
    # if len(cohort_register) > 0:
    #     patent_register_rate = (
    #         cohort_register['registerDate'].notna().mean()
    #     )
    # else:
    #     patent_register_rate = 0.0
    # ===============================
    # 등록률 코호트: t-4 ~ t-3
    # ===============================
    VALID_SUCCESS_STATUS = ['등록', '공개']
    cohort_register = df[
        (df['application_year'] >= t - 4) &
        (df['application_year'] <= t - 3)
    ]

    if len(cohort_register) > 0:
        # 전체 출원 수
        total_applications = len(cohort_register)

        # 등록 + 공개 건수
        success_count = cohort_register[
            cohort_register['registerStatus'].isin(['등록', '공개'])
        ].shape[0]

        patent_register_rate = success_count / total_applications
    else:
        patent_register_rate = 0.0


    # -------------------------------
    # 피인용 (기본)
    # -------------------------------
    patent_citation_total = (
        citation_window['피인용 횟수'].sum()
        if len(citation_window) > 0 else 0.0
    )

    patent_citation_avg = (
        patent_citation_total / max(1, patent_count_5y)
    )

    # -------------------------------
    # 🔥 연령 보정 피인용
    # -------------------------------
    if len(citation_window) > 0:
        citation_window['patent_age'] = (
            t - citation_window['application_year']
        ).clip(lower=1)

        citation_window['citation_per_year'] = (
            citation_window['피인용 횟수'] / citation_window['patent_age']
        )

        patent_citation_age_adj_total = (
            citation_window['citation_per_year'].sum()
        )
    else:
        patent_citation_age_adj_total = 0.0

    patent_citation_age_adj_avg = (
        patent_citation_age_adj_total / max(1, patent_count_5y)
    )

    # -------------------------------
    # 특허 0건 플래그
    # -------------------------------
    no_patent_flag = int(patent_count_5y == 0)

    return {
        'company_norm': company_norm,
        '연도': t,
        'patent_count_5y': patent_count_5y,
        'patent_count_recent': patent_count_recent,
        'ipc_diversity': ipc_diversity,
        'patent_register_rate': patent_register_rate,
        'patent_citation_total': patent_citation_total,
        'patent_citation_avg': patent_citation_avg,
        # 🔥 추가 피처
        'patent_citation_age_adj_total': patent_citation_age_adj_total,
        'patent_citation_age_adj_avg': patent_citation_age_adj_avg,
        'no_patent_flag': no_patent_flag
    }


In [ ]:
patent_feature_rows = []

for row in tqdm(
    company_year_df.itertuples(index=False),
    total=len(company_year_df),
    desc="특허 피처 생성"
):
    feat = make_patent_features(
        company_norm=row.company_norm,
        t=row.연도,
        patent_df=patent_df
    )

    if feat is not None:
        patent_feature_rows.append(feat)

patent_feature_df = pd.DataFrame(patent_feature_rows)


In [ ]:
patent_feature_df['연도'].unique()


- 특허 피처 엔지니어링 과정에서 롤링 윈도우의 warm-up 기간을 함수 내부에서 강제하여,
충분한 과거 데이터가 확보되지 않은 연도(2019 이전)가
학습 데이터에 포함되지 않도록 수정하였다.

## 재무 데이터와 결합

In [ ]:
financial_df['연도'] = pd.to_numeric(financial_df['연도'], errors='coerce').astype('Int64')
patent_feature_df['연도'] = pd.to_numeric(patent_feature_df['연도'], errors='coerce').astype('Int64')


In [ ]:
financial_df[['연도']].dtypes, patent_feature_df[['연도']].dtypes


In [ ]:
financial_df['매출액'].isna().mean(), (financial_df['매출액'] == 0).mean()


In [ ]:
financial_agg = (
    financial_df
    .groupby(['company_norm', '연도'], as_index=False)
    .agg({
        '기업명': 'first',
        '사업자등록번호': 'first',
        'crno': 'first',
        'dart_corp_code': 'first',
        'sectors': 'first',
        'field': 'first',

        '매출액': 'sum',
        '영업이익': 'sum',
        '당기순이익': 'sum',
        '자산총계': 'max',
        '부채총계': 'max',
        '자본총계': 'max',
        '연구개발비': 'sum',
        'CAPEX': 'sum',
        '유형자산_당기': 'max',
        '유형자산_전기': 'max',
        '유형자산_증감': 'sum'
    })
)


In [ ]:
financial_agg

In [ ]:
final_df = financial_agg.merge(
    patent_feature_df,
    on=['company_norm', '연도'],
    how='left',
    validate='one_to_one'
)


In [ ]:
len(financial_agg), len(final_df)


In [ ]:
final_df['매출액'].isna().mean()


In [ ]:
final_df

In [ ]:
print('financial_df:',financial_df.columns)
print('patent_feature_df:',patent_feature_df.columns)
print('final_df:',final_df.columns)


In [ ]:
print(len(financial_df), len(final_df))


## 타깃(y) 생성 — “t 정보로 t+1 성장 여부 예측”

In [ ]:
import numpy as np
import pandas as pd

money_cols = [
    '매출액','영업이익','당기순이익',
    '자산총계','부채총계','자본총계',
    '연구개발비','CAPEX',
    '유형자산_당기','유형자산_전기','유형자산_증감'
]

for col in money_cols:
    final_df[col] = (
        final_df[col]
        .astype(str)
        .str.replace(',', '', regex=False)
    )
    final_df[col] = pd.to_numeric(final_df[col], errors='coerce')



In [ ]:
final_df[['매출액']].head()
final_df['매출액'].isna().mean()


In [ ]:
final_df['revenue_growth'] = (
    final_df
    .groupby('company_norm')['매출액']
    .pct_change()       # pct_change를 통해서 매출 성장률 계산됨
)


In [ ]:
final_df

In [ ]:
final_df = final_df[
    (final_df['연도'] >= 2019) &
    (final_df['연도'] <= 2024)
].copy()


In [ ]:
final_df['연도'].value_counts().sort_index()


In [ ]:
# growth_flag : 같은 연도 안에서 성장률이 상위 30%면 1
final_df['growth_flag'] = (
    final_df
    .groupby('연도')['revenue_growth']
    .transform(lambda x: (x >= x.quantile(0.7)).astype(int))
)


In [ ]:
final_df.groupby('연도')['growth_flag'].mean()


In [ ]:
# y : 올해의 정보로 다음 해에 잘 성장할 기업인지 예측하기 위해서 1년 뒤로 당겨서 만듦
final_df['y'] = (
    final_df
    .groupby('company_norm')['growth_flag']
    .shift(-1)
)


In [ ]:
model_df = final_df.dropna(subset=['y']).copy()
model_df['y'] = model_df['y'].astype(int)


In [ ]:
model_df['연도'].value_counts().sort_index()


In [ ]:
model_df

In [ ]:
# 필요한 컬럼들만 남기기
keep_cols = [
    # 식별 / 시점
    'company_norm',
    '연도',
    '기업명',

    # 특허 피처
    'patent_count_5y',
    'patent_count_recent',
    'ipc_diversity',
    'patent_register_rate',
    'patent_citation_total',
    'patent_citation_avg',
    'patent_citation_age_adj_total',
    'patent_citation_age_adj_avg',
    'no_patent_flag',

    # 성장 관련
    # 'revenue_growth',
    # 'growth_flag',
    # 'y'
]

model_df = model_df[keep_cols].copy()


In [ ]:
model_df.to_csv('../../output/feature_engineering_dataset_8_patent.csv', encoding='utf-8-sig')

# 모델링
- 모델 : LogReg, RandomForest, LightBGM, XGBoost
- 검증 방식 : Holdout, GroupKFold, RollingValidation
- 평가지표: PR-AUC, Top-20/50 Precision, F1, Brier
---
> - model_df : feature + y 포함 (정상)
> - 연도 범위 : 2019–2023
> - y : t년 피처 → t+1년 성장 여부(0,1)

## X랑 y에 들어갈 df 불러오기

### feature 데이터 

In [ ]:
feat_en_df = pd.read_csv('../../data/normalized_patent_features.csv', encoding="utf-8", encoding_errors='replace', engine="python")
feat_en_df

### 정답 데이터

In [ ]:
target_df = pd.read_csv('../../data/연도별 성장여부.csv', encoding="utf-8", encoding_errors='replace', engine="python")
target_df

In [ ]:
target_long_df = target_df.melt(
    id_vars="기업명",        # 고정할 컬럼
    var_name="연도",        # 새로 생길 연도 컬럼 이름
    value_name="y"          # 타겟 변수
)

target_long_df


In [ ]:
target_long_df.to_csv('../../output/연도별_성장여부_컬럼변환.csv', encoding='utf-8-sig')

In [ ]:
# feature 데이터 연도 → int
feat_en_df["연도"] = feat_en_df["연도"].astype(int)

# target 데이터 연도 → int
target_long_df["연도"] = target_long_df["연도"].astype(int)


In [ ]:
model_df = feat_en_df.merge(
    target_long_df,
    left_on=["기업명", "target_year"],
    right_on=["기업명", "연도"],
    how="left"
)

model_df.head(3)

In [ ]:
model_df.columns.tolist()


# 왼쪽 연도  → 연도_x
# 오른쪽 연도 → 연도_y


## Feature / Label 분리

In [ ]:
# # ID / Label 컬럼
# id_cols = ['기업명', 'company_norm', '연도']    # 행을 구분하기 위한 정보
# label_cols = ['y', 'growth_flag', 'revenue_growth', 'asset_growth']

# # 숫자형 피처만 사용
# feature_cols = (
#     model_df
#     .drop(columns=id_cols + label_cols, errors='ignore')
#     .select_dtypes(include=['int64', 'float64'])
#     .columns
#     .tolist()
# )

# X = model_df[feature_cols].copy()
# y = model_df['y'].copy()
# years = model_df['연도'].copy()
# groups = model_df['company_norm'].copy()



In [ ]:
# =========================
# Feature / Label 분리 (최종)
# =========================

# ID / Label 컬럼
id_cols = ['기업명', 'company_norm', 'target_year']
label_cols = ['y']

# 숫자형 피처만 사용
feature_cols = (
    model_df
    .drop(columns=id_cols + label_cols, errors='ignore')
    .select_dtypes(include=['int64', 'float64'])
    .columns
    .tolist()
)

# 입력 / 정답
X = model_df[feature_cols].copy()
y = model_df['y'].copy()

# 검증용 보조 정보
years = model_df['target_year'].copy()
groups = model_df['company_norm'].copy()


## train/test 분리

In [ ]:
# =========================
# STEP 1. 연도 기준 분리
# =========================
train_mask = years <= 2022
test_mask  = years == 2023

X_train_full = X[train_mask]
y_train_full = y[train_mask]
groups_train = groups[train_mask]
years_train  = years[train_mask]

X_test = X[test_mask]
y_test = y[test_mask]


## 결측치 처리 함수(잠깐 실행 x)

In [ ]:
def fill_na_train_test(X_tr, X_te, feature_cols):
    patent_cols = [c for c in feature_cols if c.startswith('patent_') or c == 'no_patent_flag']
    financial_cols = [c for c in feature_cols if c not in patent_cols]

    # 특허 → 0
    X_tr[patent_cols] = X_tr[patent_cols].fillna(0)
    X_te[patent_cols] = X_te[patent_cols].fillna(0)

    # 재무 → train median
    for col in financial_cols:
        med = X_tr[col].median()
        X_tr[col] = X_tr[col].fillna(med)
        X_te[col] = X_te[col].fillna(med)

    return X_tr, X_te


## 공통 평가 함수

In [ ]:
# import numpy as np
# from sklearn.metrics import (
#     accuracy_score, roc_auc_score, f1_score,
#     average_precision_score, brier_score_loss
# )

# def top_k_precision(y_true, y_prob, k):
#     k = min(k, len(y_true))
#     idx = np.argsort(y_prob)[-k:]
#     return y_true.iloc[idx].mean()


In [ ]:
from sklearn.metrics import (
    accuracy_score,
    roc_auc_score,
    f1_score,
    average_precision_score,
    brier_score_loss
)

def evaluate_model(y_true, y_pred, y_proba):
    """
    공통 모델 평가 함수
    - 이진 분류 기준
    - 확률 예측 기반 지표 포함
    """
    return {
        "accuracy": accuracy_score(y_true, y_pred),
        "roc_auc": roc_auc_score(y_true, y_proba),
        "f1": f1_score(y_true, y_pred),
        "avg_precision": average_precision_score(y_true, y_proba),
        "brier": brier_score_loss(y_true, y_proba)
    }


## 모델 정의(Baseline)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [ ]:
models = {
    "LogReg": LogisticRegression(
        max_iter=1000,
        n_jobs=-1
    ),
    "RF": RandomForestClassifier(
        n_estimators=300,
        max_depth=8,
        random_state=42,
        n_jobs=-1
    ),
    "XGB": XGBClassifier(
        n_estimators=300,
        max_depth=5,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        eval_metric="logloss",
        random_state=42
    ),
    "LGBM": LGBMClassifier(
        n_estimators=300,
        learning_rate=0.05,
        num_leaves=31,
        random_state=42
    )
}

# ① baseline 전용
baseline_models = {
    "LogReg": models["LogReg"]
}

# ② 본 게임용 (트리 모델만)
tree_models = {
    k: v for k, v in models.items()
    if k != "LogReg"
}



## 하이퍼파라미터 튜닝

In [ ]:
from sklearn.base import clone
import itertools
import pandas as pd
import numpy as np

def rolling_hyperparam_search(
    model_class,
    param_grid,
    X, y, years,
    start_year=2019,
    end_year=2022
):
    results = []

    param_names = list(param_grid.keys())
    param_combinations = list(itertools.product(*param_grid.values()))

    for params in param_combinations:
        param_dict = dict(zip(param_names, params))
        fold_scores = []

        for train_end in range(start_year, end_year + 1):
            val_year = train_end + 1

            train_mask = years <= train_end
            val_mask   = years == val_year

            X_tr  = X.loc[train_mask]
            y_tr  = y.loc[train_mask]

            X_val = X.loc[val_mask]
            y_val = y.loc[val_mask]


            if X_tr.shape[0] == 0 or X_val.shape[0] == 0:
                continue

            if y_tr.nunique() < 2:
                continue


            model = model_class(**param_dict)
            model.fit(X_tr, y_tr)

            y_pred = model.predict(X_val)
            y_proba = model.predict_proba(X_val)[:, 1]

            score = evaluate_model(y_val, y_pred, y_proba)
            fold_scores.append(score)

        if len(fold_scores) == 0:
            continue

        avg_score = pd.DataFrame(fold_scores).mean().to_dict()
        avg_score.update(param_dict)

        results.append(avg_score)
    
    return pd.DataFrame(results)



## ML Flow

In [ ]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import RandomForestClassifier
# from xgboost import XGBClassifier
# from lightgbm import LGBMClassifier

# model_configs = {
#     "LogReg": (
#         LogisticRegression,
#         {
#             "C": [0.1, 1.0, 5.0],
#             "penalty": ["l2"],
#             "solver": ["lbfgs"],
#             "max_iter": [1000]
#         }
#     ),

#     "RF": (
#         RandomForestClassifier,
#         {
#             "n_estimators": [200, 400],
#             "max_depth": [5, 10],
#             "min_samples_leaf": [1, 3],
#             "random_state": [42],
#             "n_jobs": [-1]
#         }
#     ),

#     "XGB": (
#         XGBClassifier,
#         {
#             "n_estimators": [200, 400],
#             "max_depth": [4, 6],
#             "learning_rate": [0.03, 0.05],
#             "subsample": [0.8],
#             "colsample_bytree": [0.8],
#             "eval_metric": ["logloss"],
#             "random_state": [42]
#         }
#     ),

#     "LGBM": (
#         LGBMClassifier,
#         {
#             "n_estimators": [200, 400],
#             "learning_rate": [0.03, 0.05],
#             "num_leaves": [15, 31],
#             "max_depth": [-1, 5],
#             "random_state": [42]
#         }
#     )
# }
# 

In [ ]:
# import mlflow
# import mlflow.sklearn

# mlflow.set_experiment("growth_prediction_rolling")

# results = []

# for model_name, (model_class, param_grid) in model_configs.items():

#     print(f"\n=== {model_name} tuning ===")

#     with mlflow.start_run(run_name=f"{model_name}_rolling"):

#         df_results = rolling_hyperparam_search(
#             model_class=model_class,
#             param_grid=param_grid,
#             X=X_train_full,
#             y=y_train_full,
#             years=years_train,
#             start_year=2019,
#             end_year=2022
#         )

#         # ▶ 가장 좋은 파라미터 선택 (ROC 기준)
#         best_row = df_results.sort_values("roc_auc", ascending=False).iloc[0]

#         # ▶ 파라미터 로깅
#         best_params = {
#             k: best_row[k]
#             for k in param_grid.keys()
#         }

#         # 타입 정리 (중요!)
#         for k, v in best_params.items():
#             if isinstance(v, float) and k not in ["learning_rate"]:
#                 best_params[k] = int(v)

#         mlflow.log_params(best_params)

#         # ▶ 성능 로깅
#         mlflow.log_metrics({
#             "roc_auc": best_row["roc_auc"],
#             "f1": best_row["f1"],
#             "avg_precision": best_row["avg_precision"],
#             "brier": best_row["brier"]
#         })

#         # ▶ 결과 테이블도 아티팩트로 저장
#         result_path = f"{model_name}_rolling_results.csv"
#         df_results.to_csv(result_path, index=False)
#         mlflow.log_artifact(result_path)

#         results.append((model_name, best_params, best_row))


In [ ]:
# =========================================================
# 0. 라이브러리
# =========================================================
from datetime import datetime
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

import mlflow
import mlflow.sklearn


# =========================================================
# 1. 모델 설정
# =========================================================
model_configs = {
    "LogReg": (
        LogisticRegression,
        {
            "C": [0.1, 1.0, 5.0],
            "penalty": ["l2"],
            "solver": ["lbfgs"],
            "max_iter": [1000]
        }
    ),

    "RF": (
        RandomForestClassifier,
        {
            "n_estimators": [200, 400],
            "max_depth": [5, 10],
            "min_samples_leaf": [1, 3],
            "random_state": [42],
            "n_jobs": [-1]
        }
    ),

    "XGB": (
        XGBClassifier,
        {
            "n_estimators": [200, 400],
            "max_depth": [4, 6],
            "learning_rate": [0.03, 0.05],
            "subsample": [0.8],
            "colsample_bytree": [0.8],
            "eval_metric": ["logloss"],
            "random_state": [42]
        }
    ),

    "LGBM": (
        LGBMClassifier,
        {
            "n_estimators": [200, 400],
            "learning_rate": [0.03, 0.05],
            "num_leaves": [15, 31],
            "max_depth": [-1, 5],
            "random_state": [42]
        }
    )
}


# =========================================================
# 2. MLflow 설정
# =========================================================
mlflow.set_experiment("growth_prediction_rolling")

results = []


# =========================================================
# 3. 모델별 롤링 튜닝 + 로깅
# =========================================================
for model_name, (model_class, param_grid) in model_configs.items():

    run_ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    print(f"\n=== {model_name} rolling tuning ===")

    with mlflow.start_run(
        run_name=f"{model_name}_rolling_{run_ts}"
    ):

        # 🔹 롤링 하이퍼파라미터 탐색
        df_results = rolling_hyperparam_search(
            model_class=model_class,
            param_grid=param_grid,
            X=X_train_full,
            y=y_train_full,
            years=years_train,
            start_year=2019,
            end_year=2022
        )

        # 🔹 최고 성능 파라미터 선택 (ROC 기준)
        best_row = df_results.sort_values(
            "roc_auc", ascending=False
        ).iloc[0]

        best_params = {
            k: best_row[k]
            for k in param_grid.keys()
        }

        # 🔥 타입 정리 (int 강제)
        for k, v in best_params.items():
            if isinstance(v, float) and k not in ["learning_rate"]:
                best_params[k] = int(v)

        # 🔹 MLflow 로깅
        mlflow.log_params(best_params)

        mlflow.log_metrics({
            "roc_auc": best_row["roc_auc"],
            "f1": best_row["f1"],
            "avg_precision": best_row["avg_precision"],
            "brier": best_row["brier"]
        })

        # 🔹 결과 CSV 아티팩트 저장 (run별 독립)
        result_path = f"{model_name}_rolling_results_{run_ts}.csv"
        df_results.to_csv(result_path, index=False)
        mlflow.log_artifact(result_path)

        results.append({
            "model": model_name,
            **best_params,
            "roc_auc": best_row["roc_auc"],
            "f1": best_row["f1"],
            "avg_precision": best_row["avg_precision"],
            "brier": best_row["brier"]
        })


# =========================================================
# 4. 전체 결과 요약 테이블 (노트북 확인용)
# =========================================================
summary_df = pd.DataFrame(results).sort_values(
    "roc_auc", ascending=False
)

summary_df


## 모델별 튜닝

### LGBM

In [ ]:
from lightgbm import LGBMClassifier

lgbm_param_grid = {
    "n_estimators": [200, 400],
    "learning_rate": [0.03, 0.05],
    "num_leaves": [15, 31],
    "max_depth": [-1, 5],
    "random_state": [42]
}

lgbm_results = rolling_hyperparam_search(
    LGBMClassifier,
    lgbm_param_grid,
    X_train_full,
    y_train_full,
    years_train
)

lgbm_results.sort_values("roc_auc", ascending=False).head()


### XGBoost 

In [ ]:
from xgboost import XGBClassifier

xgb_param_grid = {
    "n_estimators": [200, 400],
    "learning_rate": [0.03, 0.05],
    "max_depth": [4, 6],
    "subsample": [0.8],
    "colsample_bytree": [0.8],
    "eval_metric": ["logloss"],
    "random_state": [42]
}

xgb_results = rolling_hyperparam_search(
    XGBClassifier,
    xgb_param_grid,
    X_train_full,
    y_train_full,
    years_train
)

xgb_results.sort_values("roc_auc", ascending=False).head()


### RF

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_param_grid = {
    "n_estimators": [200, 400],
    "max_depth": [5, 10],
    "min_samples_leaf": [1, 3],
    "random_state": [42],
    "n_jobs": [-1]
}

rf_results = rolling_hyperparam_search(
    RandomForestClassifier,
    rf_param_grid,
    X_train_full,
    y_train_full,
    years_train
)

rf_results.sort_values("roc_auc", ascending=False).head()



### LR(Logistic Regression)

In [ ]:
from sklearn.linear_model import LogisticRegression

lr_param_grid = {
    "C": [0.1, 1.0, 5.0],
    "penalty": ["l2"],
    "solver": ["lbfgs"],
    "max_iter": [1000]
}

lr_results = rolling_hyperparam_search(
    LogisticRegression,
    lr_param_grid,
    X_train_full,
    y_train_full,
    years_train
)

lr_results.sort_values("roc_auc", ascending=False).head()



### 모델별 최고 성능 비교 
- ROC-AUC 최댓값 기준

In [ ]:
best_rf   = rf_results.sort_values("roc_auc", ascending=False).iloc[0]
best_lgbm = lgbm_results.sort_values("roc_auc", ascending=False).iloc[0]
best_xgb  = xgb_results.sort_values("roc_auc", ascending=False).iloc[0]
best_lr  = lr_results.sort_values("roc_auc", ascending=False).iloc[0]


In [ ]:
compare_df = pd.DataFrame([
    {"model": "RF",   **best_rf},
    {"model": "LGBM", **best_lgbm},
    {"model": "XGB",  **best_xgb},
    {"model": "LR",  **best_lr},
])

compare_df = compare_df[
    ["model", "roc_auc", "avg_precision", "brier", "accuracy", "f1"]
]

compare_df.sort_values("roc_auc", ascending=False)

# LGBM 선정

### 최종 모델 학습, 테스트 (2023 -> 2024) 

In [ ]:
best_params = lgbm_results.sort_values("roc_auc", ascending=False).iloc[0].to_dict()

# 🔥 int여야 하는 파라미터 강제 변환
int_params = ["random_state", "seed", "n_estimators", "num_leaves", "max_depth"]

for p in int_params:
    if p in best_params and best_params[p] is not None:
        best_params[p] = int(best_params[p])

best_model = LGBMClassifier(**best_params)




best_model.fit(X_train_full, y_train_full)

y_pred_test = best_model.predict(X_test)
y_proba_test = best_model.predict_proba(X_test)[:, 1]

test_metrics = evaluate_model(y_test, y_pred_test, y_proba_test)
test_metrics


## 검증

### Hold-Out 
> - Train: 2019–2022
> - Test : 2023 (→ 2024 성장 여부)

In [ ]:
# from tqdm import tqdm

# results = []

# X_tr = X[years <= 2022].copy()
# y_tr = y[years <= 2022]

# X_te = X[years == 2023].copy()
# y_te = y[years == 2023]

# X_tr, X_te = fill_na_train_test(X_tr, X_te, feature_cols)

# for name, model in tqdm(models.items(), desc="Hold-out"):
#     model.fit(X_tr, y_tr)

#     prob = model.predict_proba(X_te)[:, 1]
#     pred = (prob >= 0.5).astype(int)

#     results.append({
#         "model": name,
#         "validation": "Holdout",
#         "Accuracy": accuracy_score(y_te, pred),
#         "ROC_AUC": roc_auc_score(y_te, prob),
#         "PR_AUC": average_precision_score(y_te, prob),
#         "Top20_P": top_k_precision(y_te, prob, 20),
#         "Top50_P": top_k_precision(y_te, prob, 50),
#         "F1": f1_score(y_te, pred),
#         "Brier": brier_score_loss(y_te, prob)
#     })


In [ ]:
def holdout_validation(model, X, y, years):
    train_mask = years <= 2021
    val_mask   = years == 2022

    X_tr, y_tr = X[train_mask], y[train_mask]
    X_val, y_val = X[val_mask], y[val_mask]

    model.fit(X_tr, y_tr)
    y_pred = model.predict(X_val)
    y_proba = model.predict_proba(X_val)[:, 1]

    return evaluate_model(y_val, y_pred, y_proba)


### GroupKFold 검증
- 같은 기업이 Train / Test 에 동시에 등장 x

In [ ]:
# from sklearn.model_selection import GroupKFold

# gkf = GroupKFold(n_splits=5)

# for name, model in tqdm(models.items(), desc="GroupKFold"):
#     scores = []

#     for tr_idx, te_idx in gkf.split(X, y, groups):
#         X_tr, X_te = X.iloc[tr_idx].copy(), X.iloc[te_idx].copy()
#         y_tr, y_te = y.iloc[tr_idx], y.iloc[te_idx]

#         X_tr, X_te = fill_na_train_test(X_tr, X_te, feature_cols)

#         model.fit(X_tr, y_tr)
#         prob = model.predict_proba(X_te)[:, 1]
#         pred = (prob >= 0.5).astype(int)

#         scores.append({
#             "Accuracy": accuracy_score(y_te, pred),
#             "ROC_AUC": roc_auc_score(y_te, prob),
#             "PR_AUC": average_precision_score(y_te, prob),
#             "Top20_P": top_k_precision(y_te, prob, 20),
#             "Top50_P": top_k_precision(y_te, prob, 50),
#             "F1": f1_score(y_te, pred),
#             "Brier": brier_score_loss(y_te, prob)
#         })

#     avg = pd.DataFrame(scores).mean().to_dict()
#     avg.update({"model": name, "validation": "GroupKFold"})
#     results.append(avg)


In [ ]:
from sklearn.model_selection import GroupKFold

def group_kfold_validation(model, X, y, groups, n_splits=5):
    gkf = GroupKFold(n_splits=n_splits)
    scores = []

    for train_idx, val_idx in gkf.split(X, y, groups):
        X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]

        model.fit(X_tr, y_tr)
        y_pred = model.predict(X_val)
        y_proba = model.predict_proba(X_val)[:, 1]

        scores.append(evaluate_model(y_val, y_pred, y_proba))

    return pd.DataFrame(scores).mean().to_dict()


### Rolling Validation

In [ ]:
# rolling_years = [2021, 2022, 2023]

# for name, model in tqdm(models.items(), desc="Rolling"):
#     scores = []

#     for test_year in rolling_years:
#         X_tr = X[years < test_year].copy()
#         y_tr = y[years < test_year]

#         X_te = X[years == test_year].copy()
#         y_te = y[years == test_year]

#         X_tr, X_te = fill_na_train_test(X_tr, X_te, feature_cols)

#         model.fit(X_tr, y_tr)
#         prob = model.predict_proba(X_te)[:, 1]
#         pred = (prob >= 0.5).astype(int)

#         scores.append({
#             "Accuracy": accuracy_score(y_te, pred),
#             "ROC_AUC": roc_auc_score(y_te, prob),
#             "PR_AUC": average_precision_score(y_te, prob),
#             "Top20_P": top_k_precision(y_te, prob, 20),
#             "Top50_P": top_k_precision(y_te, prob, 50),
#             "F1": f1_score(y_te, pred),
#             "Brier": brier_score_loss(y_te, prob)
#         })

#     avg = pd.DataFrame(scores).mean().to_dict()
#     avg.update({"model": name, "validation": "Rolling"})
#     results.append(avg)


In [ ]:
# def rolling_validation(model, X, y, years):
#     results = []

#     rolling_years = [
#         (2019, 2020),
#         (2020, 2021),
#         (2021, 2022),
#         (2022, 2023),
#     ]

#     for train_end, val_year in rolling_years:
#         train_mask = years <= train_end
#         val_mask   = years == val_year

#         X_tr, y_tr = X[train_mask], y[train_mask]
#         X_val, y_val = X[val_mask], y[val_mask]

#         model.fit(X_tr, y_tr)
#         y_pred = model.predict(X_val)
#         y_proba = model.predict_proba(X_val)[:, 1]

#         res = evaluate_model(y_val, y_pred, y_proba)
#         res["train_end"] = train_end
#         res["val_year"] = val_year
#         results.append(res)

#     return pd.DataFrame(results)

def rolling_validation(model, X, y, years):
    fold_scores = []

    for train_end in sorted(years.unique()):
        val_year = train_end + 1

        train_mask = years <= train_end
        val_mask   = years == val_year

        # ✅ pandas 기준 slicing (중요)
        X_tr  = X.loc[train_mask]
        y_tr  = y.loc[train_mask]
        X_val = X.loc[val_mask]
        y_val = y.loc[val_mask]

        # 🔥 방어 1: 샘플 수
        if X_tr.shape[0] == 0 or X_val.shape[0] == 0:
            continue

        # 🔥 방어 2: train에 클래스 2개 이상
        if y_tr.nunique() < 2:
            continue

        # 🔥 방어 3: val에 클래스 존재
        if y_val.nunique() < 1:
            continue

        model_clone = clone(model)
        model_clone.fit(X_tr, y_tr)

        y_pred  = model_clone.predict(X_val)
        y_proba = model_clone.predict_proba(X_val)[:, 1]

        fold_scores.append(
            evaluate_model(y_val, y_pred, y_proba)
        )

    if len(fold_scores) == 0:
        return None

    return pd.DataFrame(fold_scores).mean().to_dict()


## 모델별 전부 실행

In [ ]:
all_results = {}

# 🔥 트리 모델만 3종 검증
for name, model in tree_models.items():
    print(f"\n=== {name} ===")

    all_results[name] = {
        "holdout": holdout_validation(
            model, X_train_full, y_train_full, years_train
        ),
        "groupkfold": group_kfold_validation(
            model, X_train_full, y_train_full, groups_train
        ),
        "rolling": rolling_validation(
            model, X_train_full, y_train_full, years_train
        )
    }


In [ ]:
rows = []

for model_name, validations in all_results.items():
    for val_type, metrics in validations.items():

        if metrics is None:
            continue

        row = {
            "model": model_name,
            "validation": val_type
        }
        row.update(metrics)

        rows.append(row)

results_df = pd.DataFrame(rows)
results_df


In [ ]:
import mlflow
from datetime import datetime

run_ts = datetime.now().strftime("%Y%m%d_%H%M%S")

with mlflow.start_run(
    run_name=f"validation_summary_{run_ts}"
):

    # 1️⃣ 전체 결과 CSV 저장
    summary_path = f"validation_results_summary_{run_ts}.csv"
    results_df.to_csv(summary_path, index=False)
    mlflow.log_artifact(summary_path)

    # 2️⃣ 핵심 metric만 요약해서 logging (선택)
    for _, row in results_df.iterrows():
        prefix = f"{row['model']}_{row['validation']}"

        mlflow.log_metric(f"{prefix}_roc_auc", row["roc_auc"])
        mlflow.log_metric(f"{prefix}_f1", row["f1"])
        mlflow.log_metric(f"{prefix}_avg_precision", row["avg_precision"])
        mlflow.log_metric(f"{prefix}_brier", row["brier"])


In [ ]:
key_metrics = ["roc_auc", "avg_precision", "brier"]

compare_table = (
    results_df
    .pivot_table(
        index="model",
        columns="validation",
        values=key_metrics
    )
    .round(3)
)

compare_table


- ① 검증에서 고른 best_params로
- ② 2023년까지 모든 데이터로 재학습하고
- ③ 2024년 말 피처를 넣어
- ④ 2025년 성장 확률을 예측한다